In [8]:
import simpy
import random
import itertools

# fmt: off
RANDOM_SEED = 42
SIM_TIME = 1000            # Simulation time (seconds)
# fmt: on


def job(name, env, server):

    job_cpu_req = random.randint(1,10)
    job_hdd_req = random.randint(1,10)
    job_com = random.randint(1,5)
    print(f'{env.now:6.1f} s: {name} arrived at server and {job_cpu_req} CPU and {job_hdd_req} HDD requested' )
    with server.cpu.get(job_cpu_req) as mem_req, server.hdd.get(job_hdd_req) as hdd_req:
        # Request one of the gas pumps
        result = yield mem_req & hdd_req | env.timeout(300)
        if (mem_req in result) and (hdd_req in result):

            # The "actual" refueling process takes some time
            yield env.timeout(job_com)
            # Release the pump
            server.cpu.put(job_cpu_req)
            server.hdd.put(job_hdd_req)
            print(f'{env.now:6.1f} s: {name} job completed CPU level: {server.cpu.level} HDD level: {server.hdd.level}')
        elif mem_req in result:
            print(f'{env.now:6.1f} s: {name} ONLY got CPU')
            server.cpu.put(job_cpu_req)
        elif hdd_req in result:
            print(f'{env.now:6.1f} s: {name} ONLY got HDD')
            server.hdd.put(job_hdd_req)

class Server:
    
    def __init__(self, env):
        self.cpu = simpy.Container(env, init=100, capacity=100)
        self.hdd = simpy.Container(env, init=100, capacity=100)    

def jobs_generator(env, server):
    """Generate new cars that arrive at the gas station."""
    #for i in itertools.count():
    for i in range(10):
        yield env.timeout(random.randint(3,10))
        env.process(job(f'job {i}', env, server))
        

def check_server_level(env,server):
    
    while True:
        print(f'CPU level: {server.cpu.level} HDD level: {server.hdd.level}')
        yield env.timeout(10)


# Setup and start the simulation
print('Edge Computing Simulation')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()

server = Server(env)
env.process(jobs_generator(env, server))
env.process(check_server_level(env, server))

# Execute!
env.run(until=SIM_TIME)

Edge Computing Simulation
CPU level: 100 HDD level: 100
   4.0 s: job 0 arrived at server and 5 CPU and 4 HDD requested
   6.0 s: job 0 job completed CPU level: 100 HDD level: 100
   7.0 s: job 1 arrived at server and 2 CPU and 9 HDD requested
   8.0 s: job 1 job completed CPU level: 100 HDD level: 100
CPU level: 100 HDD level: 100
  12.0 s: job 2 arrived at server and 1 CPU and 1 HDD requested
  13.0 s: job 2 job completed CPU level: 100 HDD level: 100
CPU level: 100 HDD level: 100
  21.0 s: job 3 arrived at server and 4 CPU and 9 HDD requested
  26.0 s: job 3 job completed CPU level: 100 HDD level: 100
  27.0 s: job 4 arrived at server and 9 CPU and 4 HDD requested
CPU level: 91 HDD level: 96
  30.0 s: job 5 arrived at server and 4 CPU and 8 HDD requested
  32.0 s: job 4 job completed CPU level: 96 HDD level: 92
  35.0 s: job 5 job completed CPU level: 100 HDD level: 100
  39.0 s: job 6 arrived at server and 1 CPU and 3 HDD requested
CPU level: 99 HDD level: 97
  43.0 s: job 6 job co